In [1]:
import pystac
import os
import glob
import fsspec
import json

In [9]:
dslist=json.load(fsspec.open("https://eerie.cloud.dkrz.de/datasets").open())

In [2]:
intakeurl_root="https://raw.githubusercontent.com/eerie-project/intake_catalogues/refs/heads/main/dkrz/disk/stac-templates/"

In [3]:
cvurl="https://raw.githubusercontent.com/eerie-project/dreq_tools/refs/heads/update_tables/Tables/EERIE_CV.json"
cv=json.load(fsspec.open(cvurl).open())["CV"]

In [4]:
cvmodels=list(cv["source_id"].keys())

In [5]:
cwd=os.getcwd()
models=[
    a for a in glob.glob(cwd+"/*")
    if os.path.isdir(a)
]

In [16]:
def define_catalog(modelpath):
    model=modelpath.split('/')[-1]
    modelupper=model.upper()
    if not modelupper in cvmodels:
        return None
    description=cv["source_id"][modelupper]["source"]
    description+=(
        "\n\n[Imprint](https://www.dkrz.de/en/about-en/contact/impressum) "+
        "and \n[Privacy Policy](https://www.dkrz.de/en/about-en/contact/en-datenschutzhinweise).\n"
    )
    cat = pystac.Catalog(
        id="EERIE-"+modelupper,
        title="EERIE "+modelupper+" data in Zarr format",
        description=description,
        href=intakeurl_root+model+".json"
    )
    cat.add_link(
        pystac.Link(
            "parent",
            "https://swift.dkrz.de/v1/dkrz_7fa6baba-db43-4d12-a295-8e3ebb1a01ed/catalogs/stac-catalog-eeriecloud.json",
            media_type=pystac.MediaType.JSON
        )   
    )
    for entry in dslist:
        if model+'.' in entry:
            cat.add_link(
                pystac.Link(
                    pystac.RelType.ITEM,
                    "https://eerie.cloud.dkrz.de/datasets/"+entry+"/stac",
                    media_type=pystac.MediaType.JSON
                )   
            )
    return cat

In [17]:
for model in models:
    staccat=define_catalog(model)
    if staccat:
        print(model+"/stac-"+model.split('/')[-1]+".json")
        sd=staccat.to_dict()
        json.dump(sd,fsspec.open(model+"/stac-"+model.split('/')[-1]+".json","w").open())

/work/bm1344/DKRZ/intake_catalogues/dkrz/disk/stac-templates/ifs-fesom2-sr/stac-ifs-fesom2-sr.json
/work/bm1344/DKRZ/intake_catalogues/dkrz/disk/stac-templates/icon-esm-er/stac-icon-esm-er.json
